## Persistencia de modelos

Almacenar los modelos en disco una vez entrenados. El modelo contiene toda la información necesaria para ser usado en producción, es decir, para relizar predicciones sobre nuevos datos.

**Importante**: para cargar el modelo es necesaria la misma versión de scikit-learn que se usó para exportar el modelo

[Guía de usuario](https://scikit-learn.org/stable/modules/model_persistence.html)

In [1]:
# %pip install joblib
import joblib
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression

X, y = load_boston(return_X_y=True)

lr = LinearRegression()
lr.fit(X, y)
print(lr.score(X, y))

joblib.dump(lr, 'model.pkl') 

lr_new = joblib.load('model.pkl')
lr_new.score(X, y)

0.7406426641094095


0.7406426641094095

## Combinación de modelos

De manera similar a los métodos de ensemble, combinar la salida de varios modelos puede mejorar de forma notable el rendimiento

### Voting

Con esta técnica combinamos la predicción de los modelos individuales para generar una predicción final. En el caso de **clasificación** tenemos dos formas de combinar las predicciones:

  * `hard voting`, la predicción final se calcula como la clase que predice la mayoría de los modelos individuales
  
  * `soft voting`, la predicción final se calcula como la clase más probable después de sumar las probabilidades de los modelos individuales. Además, cada modelo puede tener un peso diferente

Para **regresión** la predición final se calcula como la media de las predicciones individuales. Cada modelo puede tener un peso distinto, de forma que se calcula la media ponderada.

[Voting](https://scikit-learn.org/stable/modules/ensemble.html#voting-classifier)



In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

X, y = load_breast_cancer(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

clf1 = LogisticRegression(solver='liblinear')
clf2 = SVC()
clf3 = RandomForestClassifier()

vot_clf = VotingClassifier(estimators=[('lr', clf1), ('svc', clf2), ('rfc', clf3)], voting='hard')

vot_clf.fit(X_train, y_train)
print(f'Voting classifier: {vot_clf.score(X_test, y_test):.3f}')

for clf in (clf1, clf2, clf3):
    clf.fit(X_train, y_train)
    print(f'{clf.__class__.__name__}: {clf.score(X_test, y_test):.3f}')

Voting classifier: 0.986
LogisticRegression: 0.958
SVC: 0.937
RandomForestClassifier: 0.972


También podemos ajustar los parámetros de los modelos individuales usando validación cruzada:

In [3]:
from sklearn.model_selection import GridSearchCV

params = {'lr__C': [1.0, 10, 100.0], 
          'rfc__n_estimators': [20, 200], 
          'svc__C': [1, 10, 100]}

grid = GridSearchCV(estimator=vot_clf, param_grid=params, cv=5)
grid = grid.fit(X_train, y_train)

### Stacking

Método para combinar modelos. Las predicciones de los modelos individuales se usan como entrada para otro meta-modelo, que aprende a combinarlas (*generalizer* en la figura):

<img src=../../img/stacking.png width=600>

Es similar al método *voting* con pesos, pero en lugar de estar fijos se aprenden a partir de los datos. Además el meta-modelo podría ser no lineal.

[Fuente](https://www.commonlounge.com/discussion/1697ade39ac142988861daff4da7f27d)

[Stacking](https://scikit-learn.org/stable/modules/ensemble.html#stacked-generalization)

In [4]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.linear_model import RidgeCV, LinearRegression
from sklearn.svm import SVR

X, y = load_boston(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

estimators = [('rr', RidgeCV()),
              ('svr', SVR()),
              ('rfr', RandomForestRegressor())]

sr = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
sr.fit(X_train, y_train)
sr.score(X_test, y_test)

0.7912865416591925

Las clases `VotingClassifier`, `VotingRegressor`, `StackingClassifier` y `StackingRegressor` se pueden usar como cualquier otro modelo. Tienen métodos para predecir y se pueden usar dentro de `GridSearchCV` para buscar los parámetros óptimos de cada modelo.